In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf

In [2]:
df = pd.read_csv(r"C:\Abin\Mobile_Network_Intrusion_Data.csv")
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [3]:
#Changing the form of the dependent variable
def num_trans(ctgry):
    if ctgry=='normal':
        return 0
    elif ctgry=='anomaly':
        return 1

In [4]:
#Calling the defined function
df['ctgry'] = df['class'].apply(num_trans)

In [6]:
#Forming independent and dependent variables
X = df.drop(['class','ctgry'],axis=1)
y = df.loc[:,'ctgry']

In [8]:
#Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [16]:
#Tensorflow variables for Deep Neural Network Implementation
featur = [tf.feature_column.numeric_column('logged_in'),
            tf.feature_column.numeric_column('src_bytes'),
            tf.feature_column.numeric_column('dst_bytes'),
            tf.feature_column.numeric_column('land'),
            tf.feature_column.numeric_column('wrong_fragment'),
            tf.feature_column.numeric_column('urgent'),
            tf.feature_column.numeric_column('count'),
            tf.feature_column.numeric_column('num_failed_logins'),
            tf.feature_column.numeric_column('serror_rate'),
            tf.feature_column.numeric_column('num_compromised'),
            tf.feature_column.numeric_column('root_shell'),
            tf.feature_column.numeric_column('dst_host_count'),
            tf.feature_column.numeric_column('su_attempted'),
            tf.feature_column.numeric_column('num_root'),
            tf.feature_column.numeric_column('num_file_creations'),
            tf.feature_column.numeric_column('num_shells'),
            tf.feature_column.numeric_column('num_access_files'),
            tf.feature_column.numeric_column('duration'),
            tf.feature_column.numeric_column('diff_srv_rate'),
            tf.feature_column.numeric_column('is_guest_login'),
            tf.feature_column.numeric_column('is_host_login'),
            tf.feature_column.numeric_column('dst_host_srv_rerror_rate'),
            tf.feature_column.numeric_column('dst_host_srv_diff_host_rate'),
            tf.feature_column.numeric_column('srv_serror_rate'),
            tf.feature_column.numeric_column('rerror_rate'),
            tf.feature_column.numeric_column('srv_rerror_rate'),
            tf.feature_column.numeric_column('same_srv_rate'),
            tf.feature_column.numeric_column('srv_diff_host_rate'),
            tf.feature_column.numeric_column('srv_count'),
            tf.feature_column.numeric_column('dst_host_rerror_rate'),
            tf.feature_column.numeric_column('dst_host_same_srv_rate'),
            tf.feature_column.numeric_column('dst_host_diff_srv_rate'),
            tf.feature_column.numeric_column('dst_host_same_src_port_rate'),
            tf.feature_column.numeric_column('num_outbound_cmds'),
            tf.feature_column.numeric_column('dst_host_serror_rate'),
            tf.feature_column.numeric_column('dst_host_srv_serror_rate'),
            tf.feature_column.numeric_column('dst_host_srv_count'),
            tf.feature_column.numeric_column('hot')
           ]

In [17]:
#Artificial Neural Network model parameters
def inpt(num_epochs,num_btchs,shuffle):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X_train,
        y=y_train,
        batch_size=num_btchs,
        shuffle=shuffle,
        num_epochs=num_epochs
    )

In [18]:
#Artificial Neural Network  evaluation
def evaltn(num_epochs,num_btchs,shuffle):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X_test,
        y=y_test,
        batch_size=num_btchs,
        shuffle=shuffle,
        num_epochs=num_epochs
    )

In [19]:
#Creating the Artificial Neural Network Classifier
ann_mdl = tf.estimator.DNNClassifier(n_classes=3,
                                       optimizer='Adam',
                                       hidden_units=[1024,512,256,32,3],
                                       feature_columns=featur,
                                       activation_fn=tf.nn.relu,
                                       dropout=0.30
                                      )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Abin\\AppData\\Local\\Temp\\tmpvgapvv2d', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
#Training the Artificial Neural Network model
ann_mdl.train(input_fn=inpt(100,128,True),steps=500)



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Abin\AppData\Local\Temp\tmpvgapvv2d\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 63.34705, step = 0
INFO:tensorflow:global_step/sec: 52.768
INFO:tensorflow:loss = 1.01

In [22]:
#Output parameters of the Artificial Neural Network model
ann_mdl.evaluate(input_fn=evaltn(100,128,True),steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-02-26T12:10:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Abin\AppData\Local\Temp\tmpvgapvv2d\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [50/500]
INFO:tensorflow:Evaluation [100/500]
INFO:tensorflow:Evaluation [150/500]
INFO:tensorflow:Evaluation [200/500]
INFO:tensorflow:Evaluation [250/500]
INFO:tensorflow:Evaluation [300/500]
INFO:tensorflow:Evaluation [350/500]
INFO:tensorflow:Evaluation [400/500]
INFO:tensorflow:Evaluation [450/500]
INFO:tensorflow:Evaluation [500/500]
INFO:tensorflow:Inference Time : 3.92308s
INFO:tensorflow:Finished evaluation at 2023-02-26-12:10:50
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.52867186, average_loss = 0.8866007, global_step = 500, loss = 0.8866007
INFO:tensorflow:Saving 'checkpoint_path' s

{'accuracy': 0.52867186,
 'average_loss': 0.8866007,
 'loss': 0.8866007,
 'global_step': 500}

In [26]:
#Implementing a blend of Artificial Neural Network and Linear Classifier model
ann_linear_model = tf.estimator.DNNLinearCombinedClassifier(n_classes=3,dnn_optimizer='Adam',dnn_dropout=0.30,dnn_hidden_units=[1024,512,256,32,3],dnn_feature_columns=featur,dnn_activation_fn='relu',linear_feature_columns=featur)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Abin\\AppData\\Local\\Temp\\tmpdcexs_ia', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
#Training the blend of Artificial Neural Network and Linear Classifier model
ann_linear_model.train(input_fn=inpt(100,128,True),steps=500)

INFO:tensorflow:Calling model_fn.


C:\Users\Abin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Abin\AppData\Local\Temp\tmpdcexs_ia\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 138.8479, step = 0
INFO:tensorflow:global_step/sec: 48.3344
INFO:tensorflow:loss = 0.9094062, step = 100 (2.070 sec)
INFO:tensorflow:global_step/sec: 62.4512
INFO:tensorflow:loss = 104.9591, step = 200 (1.600 sec)
INFO:tensorflow:global_step/sec: 64.0799
INFO:tensorflow:loss = 4.981284, step = 300 (1.562 sec)
INFO:tensorflow:global_step/sec: 62.8321
INFO:tensorflow:loss = 0.5865209, step = 400 (1.592 

In [28]:
#Output parameters of blend of Artificial Neural Network and Linear Classifier model
ann_linear_model.evaluate(input_fn=evaltn(100,128,True),steps=500)

INFO:tensorflow:Calling model_fn.


C:\Users\Abin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-02-26T13:03:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Abin\AppData\Local\Temp\tmpdcexs_ia\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [50/500]
INFO:tensorflow:Evaluation [100/500]
INFO:tensorflow:Evaluation [150/500]
INFO:tensorflow:Evaluation [200/500]
INFO:tensorflow:Evaluation [250/500]
INFO:tensorflow:Evaluation [300/500]
INFO:tensorflow:Evaluation [350/500]
INFO:tensorflow:Evaluation [400/500]
INFO:tensorflow:Evaluation [450/500]
INFO:tensorflow:Evaluation [500/500]
INFO:tensorflow:Inference Time : 4.52092s
INFO:tensorflow:Finished evaluation at 2023-02-26-13:03:22
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.8725156, average_loss = 14.331543, global_step = 500, loss = 14.331543
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: C:\User

{'accuracy': 0.8725156,
 'average_loss': 14.331543,
 'loss': 14.331543,
 'global_step': 500}